<a href="https://colab.research.google.com/github/sdbrgo/PERCEUL/blob/main/PERCEUL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib

# pipeline
from sklearn.pipeline import Pipeline

# preprocesing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin # used to define NumericSelector(), which is used in preprocessing

# dimensionality reduction
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# cluster validation
from sklearn.metrics import silhouette_score

# clustering
from sklearn.cluster import KMeans, DBSCAN

# Set Up Hugging Face & Mount GDrive

In [ ]:
# Hugging Face
from huggingface_hub import login
login()

# mount GDrive
from google.colab import drive
drive.mount('/content/drive')

# Import Dataset

In [ ]:
ds_name = ""
df = pd.read_csv(ds_name)

# Preprocessing

In [ ]:
#=====================================================================
# a custom and dynamic function for selecting numeric columns only.
# will be used to make the pipeline
class NumericSelector(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        self.numeric_cols_ = X.select_dtypes(include=[float, int]).columns
        return self

    def transform(self, X):
        X_num = X[self.numeric_cols_]
        return X_num
#=====================================================================
si = SimpleImputer(strategy='median')
df_i = si.fit_transform(df)
df_i = pd.DataFrame(df_i, columns=df.columns)

ss = StandardScaler()
df_i_ss = ss.fit_transform(df_i)
df_p = pd.DataFrame(df_i_ss, columns=df.columns)

# Dimensionality Reduction 1
> The best t-SNE in the iterations should be saved at the end to be used in the pipeline.
> Revise loop content to use actual dataset (current is still templated).

In [ ]:
#=====================================================================
# a function that computes for the appropriate perplexity
def choose_perplexity(n_features):
  if n_features < 10:
      return [20, 30, 40]
  elif n_features < 30:
      return [10, 20, 30]
  elif n_features < 100:
      return [5, 10, 20]
  else:
      return [5, 10]
#=====================================================================

# get number of features from dataset
n_features = df_p.shape[1]
perplexities = choose_perplexity(n_features)

# run t-SNE for each perplexity and plot
plt.figure(figsize=(15, 5 * len(perplexities)))

for i, p in enumerate(perplexities):
    tsne = TSNE(
        n_components=2,
        perplexity=p,
        n_iter=1000,
        random_state=42
    )

    df_p_embedded = tsne.fit_transform(df_p)

    plt.subplot(len(perplexities), 1, i + 1)
    plt.scatter(df_p_embedded[:, 0], df_p_embedded[:, 1], s=8)
    plt.title(f"t-SNE embedding (perplexity = {p})")
    plt.xlabel("Component 1")
    plt.ylabel("Component 2")

plt.tight_layout()
plt.show()

# Cluster Exploration


# Final Clustering

# Cluster Interpretation

# Creating the PERCEUL Pipeline